<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

# Описание данных
<br>
Набор данных находится в файле /datasets/insurance.csv. Скачать датасет.
<br>
Признаки: пол, возраст и зарплата застрахованного, количество членов его семьи.
<br>
Целевой признак: количество страховых выплат клиенту за последние 5 лет.

## Загрузка данных

Импорт необходимых библиотек

In [1]:
import numpy as np

import pandas as pd

from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings('ignore')

Загрузка данных и вывод на экран.

In [2]:
try:  
    data = pd.read_csv('C:/projects/3/insurance.csv')
except OSError as e:
    data = pd.read_csv('/datasets/insurance.csv')
display(data.head())
display(data.info())

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


None

Проверка на наличие NaN и артефактов.

In [3]:
def data_info(data):
    for column in data.columns:
        print('Значения столбца', column)
        print(data[column].sort_values().unique())
        print('Количество NaN:', data[column].isna().sum())

In [4]:
data_info(data)

Значения столбца Пол
[0 1]
Количество NaN: 0
Значения столбца Возраст
[18. 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30. 31. 32. 33. 34. 35.
 36. 37. 38. 39. 40. 41. 42. 43. 44. 45. 46. 47. 48. 49. 50. 51. 52. 53.
 54. 55. 56. 57. 58. 59. 60. 61. 62. 65.]
Количество NaN: 0
Значения столбца Зарплата
[ 5300.  6000.  7400.  8900.  9800. 10000. 10600. 10800. 11000. 11200.
 11300. 12200. 12900. 13000. 13200. 13300. 13400. 13500. 13800. 13900.
 14100. 14300. 14400. 14500. 14600. 14700. 15000. 15100. 15200. 15600.
 15700. 15900. 16000. 16200. 16300. 16400. 16500. 16600. 16700. 17000.
 17100. 17300. 17400. 17500. 17600. 17700. 17800. 17900. 18100. 18200.
 18300. 18400. 18600. 18700. 18800. 18900. 19000. 19100. 19200. 19300.
 19400. 19600. 19700. 19900. 20000. 20100. 20200. 20300. 20400. 20500.
 20600. 20700. 20800. 20900. 21000. 21100. 21200. 21300. 21400. 21500.
 21600. 21700. 21800. 21900. 22000. 22100. 22200. 22300. 22500. 22600.
 22700. 22800. 22900. 23000. 23100. 23200. 23300. 23400. 23

**Вывод**
<br>
NaN и артефакты в столбцах отсутствуют.
<br>
Необходимо привести названия столбцов к змеиному регистру и заменить на латиницу.
<br>
Столбец "Возраст" и "Зарплата" привести тип данных к int.


In [5]:
#переименование столбцов
data.rename(columns={'Пол': 'gender', 'Возраст': 'age',
                   'Зарплата': 'salary',
                   'Члены семьи': 'family_member',
                   'Страховые выплаты': 'payments'
                  }, inplace=True)

#замена типа данных
data['age'] = data['age'].astype(int)
data['salary'] = data['salary'].astype(int)

#избавление от дубликатов
data= data.drop_duplicates()

display(data.head())
display(data.info())

,gender,age,salary,family_member,payments
0,1,41,49600,1,0
1,0,46,38000,1,1
2,0,29,21000,0,0
3,0,21,41700,2,0
4,1,28,26100,0,0


<class 'pandas.core.frame.DataFrame'>
Int64Index: 4847 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   gender         4847 non-null   int64
 1   age            4847 non-null   int64
 2   salary         4847 non-null   int64
 3   family_member  4847 non-null   int64
 4   payments       4847 non-null   int64
dtypes: int64(5)
memory usage: 227.2 KB


None

**Вывод**
<br>
Данные были загружены, изучены и подготовлены к решению поставленной задачи.

## Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)



Предсказания:

$$
a = X*w
$$

Задача обучения:

$$
w = \arg\min_w *MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^TX)^{-1}X^Ty
$$


Пусть $X1$ - это матрица $X$, умноженная на обратимую матрицу $P$:

$$
X1 = XP
$$

Подставим $X1$ в формулу обучения ($w1$):

$$
w1 = ((XP)^T(XP))^{-1}(XP)^Ty
$$

Проведем преобразования:

$$
w1 = (P^TX^TXP)^{-1}*(P^TX^T)*y
$$

$$
w1 = (P^T(X^TX)P)^{-1}*(P^TX^T)*y
$$

$$
w1 = P^{-1}(X^TX)^{-1}*(P^T)^{-1}*(P^TX^T)*y
$$

$$
w1 = P^{-1}*(X^TX)^{-1}*E(X^T)*y = P^{-1}(X^TX)^{-1}*(X^T)*y
$$

$$
w1 = P^{-1}(X^TX)^{-1}(X^T)*y = (P^{-1})*w
$$

$$
a = X*w
$$

$$
a1 = X1*w1 = XPP^{-1}*w
$$

$$
a1 = XPP^{-1}w = XEw = Xw = a
$$

После преобразований видно, что умножение признаков на обратимую матрицу не влияет на качество линейной регрессии.
<br>
Определим как связаны параметры линейной регрессии в исходной задаче и после преобразования.
<br>
Из вычислений выше можно вывести следующее:

$$
w1 = P^{-1}(X^TX)^{-1}(X^T)y = (P^{-1})*w
$$

Из этого следует, что параметры линейной регрессии в исходной задаче и в преобразованной связаны следующим образом:

$$
w1 =(P)^{-1}*w
$$

**Вывод**
<br>

При умножении признаков на обратимую матрицу - качество линейной регрессии не изменяется.
<br>

Параметры линейной регрессии в исходной задаче и в преобразованной изменятся следующим образом: Для новых признаков поменяются веса. Необходимо будет переобучить модель на новых признаках.


## Алгоритм преобразования

**Алгоритм**

Для защиты персональных данных клиентов умножим матрицу состояющую из признаков на обратимую матрицу.

Т.к. обратная матрица может быть только квадратной, то должна быть квадратной с размером, равным количеству признаков, а именно - 4.

**Обоснование**

Сравним разность предсказаний по текущим значениям признаков и по умноженным на обратимую матрицу.

In [6]:
#рандомная матрица 4х4
random_mat = np.random.randint(100,size = (4, 4))

#выделим тренировочную и валидационную выборку
target = data['payments']
features = data.drop('payments',axis=1)

#проведем масштабирование с помощью StandardScaler()
scaler = StandardScaler()
scaler.fit(features)

#сохраним масштабированные признаки в переменную
scaled = scaler.transform(features)

Рассчитаем веса $w$ для исходных признаков (масштабированных) по стандартной формуле $w = (X^T X)^{-1} X^T y$.

In [7]:
w = np.linalg.inv(scaled.T.dot(scaled)).dot(scaled.T).dot(target)

Рассчитаем вектор предсказаний по исходным признакам по формуле  𝑎=𝑋𝑤 .

In [8]:
a = scaled @ w

Проведем преобразование признаков.

In [9]:
scaled_new = scaled @ random_mat

Расчитаем веса $𝑤1$ для преобразованных признаков по стандартной формуле $𝑤=(𝑋𝑇𝑋)−1𝑋𝑇𝑦$.

In [10]:
w1 = np.linalg.inv(scaled_new.T.dot(scaled_new)).dot(scaled_new.T).dot(target)

Расчитаем вектор предсказаний по преобразованным признакам по формуле  𝑎=𝑋𝑤 .

In [11]:
a1 = scaled_new @ w1

Вычислим разницу.

In [12]:
diff = a - a1
display(diff.sum())

-3.8712956451636416e-14

**Вывод**
<br>

Небольшая сумма расхождений говорит о том, что ею можно принебречь , это значит, что 𝑎 = 𝑎1. Также не приводит к ухудшению качества модели - умножение признаков на обратную матрицу.


## Проверка алгоритма

Сравним две модели линейной регрессии - с текущими значениями признаков и с признаками, умноженными на обратную матрицу.

In [13]:
model = LinearRegression(normalize = True)
model.fit(features, target)
predictions = model.predict(features)
mse = mean_squared_error(target, predictions)
print('MSE Score:', mse)
print('R2_Score на исходных данных: ',r2_score(target, predictions))

MSE Score: 0.1252726382276536
R2_Score на исходных данных:  0.4302010046633359


In [14]:
features_mat = scaled @ random_mat
model.fit(features_mat, target)
predictions_mat = model.predict(features_mat)
mse = mean_squared_error(target, predictions)
print('MSE Score:', mse)
print('R2_Score для преобразованных  признаков: ',r2_score(target, predictions_mat))

MSE Score: 0.1252726382276536
R2_Score для преобразованных  признаков:  0.4302010046633359


**Вывод**
<br>
Параметры MSE Score и R2_Score  оказались одинаковыми для двух моделей линейной, поэтому можно сказать, что предложенный алгоритм может защитить данные пользователей и при этом не ухудшить качество модели машинного обучения.

# Общий вывод

<br>

В данном проекте было сделано следующее:

- Данные загружены, изучены и подготовлены к работе
- Было проверено утверждение "Изменится ли качество линейной регрессии, если умножить признаки на обратимую матрицу?"
- Был создан алгоритм для преобразования данных, благодаря которому будет сложно восстановить персональную информацию о клиентах.